<h1><center>AM-E207 Final Project</center></h1>
<h2><center>
    Infovae: Balancing learning and inference in variational autoencoders<br><br>
    Shengjia Zhao, Jiaming Song, Stefano Ermon
    </center></h2>
<h3><center>
    Contributors: Daniel J. Drennan, Katrine Pertsovski<br>    
    December 19, 2019
</center></h3>

#### Note
* There is no code in this submission. I couldn't get this working without using keras or the tensorflow source from the original authors. I've described the paper, but the coding was beyond my ability.

* I partnered with Katrine Pertsovski early on and we agreed to work on this project together. We have not spoken since November, after I had to cancel a scheduled meeting with teaching staff during business hours due to an unplanned work commitment. We discussed much of the original work, and I've left her as a collaborator, but I'm not sure if she is still enrolled in 

* 

**<h3><center>Summary</center></h3>**

Variational autoencoders (VAEs) are a family of generative models which are useful for learning complex data distributions, such as image and text recognition and generation.
By assuming samples $x_n$ depend on continuous latent features, $z_n$, we can efficiently learn an approximating distribution for the posterior distribution, $p(z | x) \propto p(z)p(x|z)$, via optimization methods.
However, variational approximating distributions are often significantly different from the true posterior we aim to learn and the variational lower bound inherently prefers learning the data distribution over the latent distribution.
Recent efforts to improve performance have incorporated regularization terms or information-theoretic expressions into the objective function to balance feature learning with data fitting.
In this project, we review a paper which generalizes many classes of VAE functions by adding multiple regularization terms to the variational objective function.
The proposed objective function gives 3 new design choices: a scale parameter for weighting the reconstruction term, a scale parameter for weighing mutual information in the model, and different divergence measures from just the KL divergence.

# Outline

1. What are generative models?
2. The VAE model
3. Problems with VAEs<br>
    3.1. Accurate inference<br>
    3.2. Information preference<br>
4. InfoVAE
5. Conclusions
6. References

# 1. What are generative models?


Generative models are directed graphs of joint distributions which are used to represent complex data, such as images and natural language (Zhao et al. 2017). Common examples of generative models are variational autoencoders (Doersch 2016) and various families of adversarial networks and adversarial autoencoders (Goodfellow et al. 2014; Makhzani et al. 2015). We discuss variational autoencoders, which are built from the assumption that there is a latent feature space we can learn as the posterior distribution, $p(z | x) \propto p(x, z)$, where $x$ is a training sample and $z$ is a latent feature with prior $p(z)$.

Kingma and Welling (2013) introduced variational autoencoders (VAEs) to increase efficiency of inference and scalability of computations when doing variational Bayes with neural networks. The authors' goals in the paper were to provide
1. an efficient method for estimating network parameters, $\theta$;
2. a function for approximating posterior inference of a latent variable given an observed training sample (amortized inference);
3. and a computationally efficient procedure for generating new samples based on the training data.
In addition to satisfying these aims, the paper introduces the "reparameterization trick", which has become a popular method when optimizing VAE models.

Recent work has shown that VAEs do not use latent information efficiently (Chen et al. 2016). Attempts to resolve this conflict have included use of more flexible variational families, along with regularization techniques such as variational dropout (Kingma, Salimans, Welling 2016). Zhao et al. propose a solution to many of these problems by generalizing the ELBO function used when fitting VAEs by introducing new scaling terms and information-theoretic techniques (Zhao et al. 2017). 

Next we present a canonical definition of VAE, given by Zhao et al., and summarizing the work in Kingma and Welling (2013). Then we walk through problems with VAEs and discuss solutions proposed for solving them. Finally, we describe the InfoVAE objective function, which generalizes the traditional ELBO by introducing new regularization factors for controlling ELBO behavior.

# 2. The VAE model


Let $x = \{x_n\}_{n=1}^{N}$ be a set of training samples given latent features $z = \{ z_n \}_{n=1}^{N}$ with prior distribution $p( z )$. Suppose we can condition the samples, $x$, on the latent features to obtain the joint—or generative—distribution $p(x, z) = p(z)p_\theta(x | z)$, where $p_\theta(x | z)$. We conventionally code $p_\theta(x | z)$ as a neural network, and the goal becomes learning the posterior distribution <br>
\begin{equation}
    p(z | x) = \frac{p_\theta(x, z)}{p(z)}. \tag{1}
\end{equation}<br>
By introducing a variational distribution $q_\phi(z | x)$, we can seek to approximate the posterior $p_\theta(z | x)$ using the ELBO function

$$
    ELBO(x) = -D_{KL}[q_\phi(z | x)\ ||\ p(z)] + \mathbb{E}_{z\sim q_\phi(z | x)}[\log p_\theta(x | z)] \leqslant \log p_\theta(x).
$$

Finally, assuming an empirical distribution $p_\mathcal{D}(x)$, we can write the complete variational family as $q_\phi(x,z) = p_\mathcal{D}(x)q_\phi(z | x)$. This allows us to average over the data to obtain the optimization objective

\begin{equation}
    ELBO
    = \mathbb{E}_{p_\mathcal{D}(x)}[ELBO(x)]
    = -\underset
    {\text{reconstruction}}
    {\underbrace{\mathbb{E}_{p_\mathcal{D}(x)}\big[D_{KL}[q_\phi(z | x)\ ||\ p(z)]\big]}}
     + \underset
     {\text{regularization}}
     {\underbrace{\mathbb{E}_{p_\mathcal{D}(x)}\big[\mathbb{E}_{z\sim q_\phi(z | x)}[\log p_\theta(x | z)]\big]}}.\tag{2}
\end{equation}

Multiple papers have shown problems with VAEs inherently stem from $(2)$, with regularization and other approaches proposed as solutions to ameliorate its limitations. It turns out that adding multiple regularization parameters, along with a factor to increase the importance of mutual information, allows us to generalize many of the models proposed up to now.

# 3. Problems with VAEs


Zhao et al. point out two problems with the classic ELBO in Equation $(2)$. First, $(2)$ can be maximized without learning an accurate representation of the variational posterior, $q_\phi(z | x)$, leading to poor proposals when generating data. Second, a complex decoding distribution may neglect the latent features we assume the conditional models depend on. We summarize the first problem as being one of accurate inference, and the second as one of information preference.

## 3.1. Accurate inference


Write the variational lower bound $(2)$ as

\begin{equation}
    ELBO =
    -\underset
    {\text{data/model minimization}}
    {\underbrace{D_{KL}[p_\mathcal{D}(x)\ ||\ p_\theta(x)]}}
    -\underset
    {\text{posterior minimization}}
    {\underbrace{\mathbb{E}_{p_\mathcal{D}(x)}\big[D_{KL}[q_\phi(z | x)\ ||\ p_\theta(z | x)]\big]}}
    \tag{3}
\end{equation}

(Equation $(14)$ in Hoffman and Johnson (2017)). Hoffman and Johnson point out that we can minimize the average difference between distributions in the first term without truly maximizing the ELBO. Also notice that the two terms are in conflict: the first is minimizing the divergence between the empirical distribution and the marginal model distribution, but the second term is minimizing the ***expected divergence*** between the variational approximation and the true posterior.

Moreover, pathological cases have been shown to exist where the ELBO is maximized without learning an accurate variational posterior $q_\phi(z | x)$, or where imbalanced, high-dimensional data spaces are overfit instead of learning the latent feature space. In these cases, a large ELBO value (small KL divergence) would suggest the model has been learned, however the model performs inference tasks poorly. Intuitively this makes sense: if the sample space, say, $\mathcal{X}$, has order which dominates the latent space, $\mathcal{Z}$, then minimizing the divergence of $\mathcal{X}$ in both models will maximize the ELBO, but minimizing the divergence of the latent feature distributions will be relatively small in comparison.

## 3.2. Information preference


Chen et al. (2016) discuss the properties of the ELBO function which make it prefer overfitting data to learning latent features. There, they suggest as one alternative starting with an approximate inference that more closely matches the posterior distribution to reduce the penalty in the cost function when starting to fit a model. Particularly for recurrent patterns in text, they suggest introducing a more lossy approximating distribution to solve the Bits-back coding efficiency problem. While this improves the use of information in the model, it comes at the cost of computing over an autoregressive model with recursion.

It would be preferable to balance information sources in the objective function using a correlation-like structure. A generalized concept of correlation is ***Mutual Information***, which we define next.

---

**Definition** (*Mutual Information*). Let $X$ and $Z$ be random variables with joint distribution $p(X,Z)$. The ***mutual information*** between $X$ and $Z$, $I(X; Z)$, is

$$
    I(X; Z) \equiv D_{KL}[p(X,Z)\ ||\ p(X)p(Z)].
$$

---

The mutual information between two variables is the KL divergence between the joint distribution and the marginal distributions of its corresponding variables. If we can factor a joint distribution into its marginal distributions perfectly, then it will have divergence zero. An example illustrating mutual information follows.

Consider a bivariate Gaussian distribution of independent standard normal random variables, $X$ and $Y$. The joint distribution is

\begin{align}
    p(x,y)
    &= \frac{1}{2\pi}\exp\left\{ -\frac{1}{2}(x^2 + y^2) \right\} = \frac{1}{2\pi}e^{-0.5x^2}e^{-0.5y^2} \\
    &= \frac{1}{\sqrt{2\pi}}e^{-0.5x^2}\frac{1}{\sqrt{2\pi}}e^{-0.5y^2} = p(x)p(y),
\end{align}

showing joint distribution factorizes (And is equivalent to the mean field assumption). Thus, the mutual information of the bivariate Gaussian distribution is zero. It is fairly straightforward to generalize this to any bivariate Gaussian with zero correlation. 

Since this is a KL divergence term, it should be easy to integrate into the ELBO of a VAE model. A regularized mutual infromation term turns out to be a factor in the InfoVAE objective.

# 4. InfoVAE


Now we introduce the InfoVAE objective function. By applying a scaling parameter, $\lambda$, to the divergence between $q_\phi(z)$ and the prior $p(z)$, and introducing a new term, $\alpha I(x; z)$, we obtain

\begin{equation}
    \mathcal{L}_{\text{InfoVAE}} =
    - \underset{\text{latent scaled divergence}}{\underbrace{\lambda D_{KL}[ q_\phi(z)\ ||\ p(z) ]}} \
     - \underset{\text{averaged encoder-posterior divergence}}{\underbrace{\mathbb{E}_{q(z)}\big[D_{KL}[q_\phi(z | x)\ ||\ p_\theta(z | x)]\big]}} \
    + \underset{\text{scaled mutual information}}{\underbrace{\alpha I(x; z)}}.
\tag{4}
\end{equation}

Equivalently,

\begin{equation}
    \mathcal{L}_{\text{InfoVAE}} =
    \underset{\text{Optimize using reparameterization trick}}{\underbrace{
    \mathbb{E}_{p_\mathcal{D}(x)}\mathbb{E}_{q_\phi(z|x)}[\log p_\theta(x | z)]
    -(1-\alpha)\mathbb{E}_{p_\mathcal{D}(x)}D_{KL}[q_\phi(z|x)\ ||\ p(z)]}}
    -\underset{\text{Choose divergence measure}}{\underbrace{(\alpha + \lambda - 1)D_{KL}[q_\phi(z)\ ||\ p(z)]}}.
    \tag{5}
\end{equation}

Reviewing $(5)$, we see 3 new design choices when building a VAE: we can set $\lambda$ to choose the importance of the divergence between the two latent variable distributions, $\alpha$ for the importance of the mutual information, and a (strict) divergence measure which is different from the KL divergence. A divergence is strict if $D[q(\cdot)\ ||\ p(\cdot)] = 0$ if and only if $q(\cdot) = p(\cdot)$.

The first two regularized terms can be optimized using gradients with the reparameterization trick. The last term can be estimated using likelihood-free techniques by sampling $x \sim p_\mathcal{D}$ and then sampling $z \sim q_\phi(z | x)$ (one procedure for doing this was presented in Goodfellow et al. 2014); alternatively, we can change the divergence criteria that is more easily optimized over. The authors show the following proposition to be true in the appendices of their paper.

---

**Proposition.** Let $\mathcal{X}$ and $\mathcal{Z}$ be continuous spaces. Suppose $\alpha < 1$ and $\lambda > 0$. Fix $I_0 = I(x; z)$ at some bounded value. Then $\mathcal{L}_{\text{InfoVAE}}$ is globally optimized if $p_\theta(x) = p_\mathcal{D}(x)$ and $q_\phi(z | x) = p_\theta(z | x)$ for all $z \in \mathcal{Z}$. 

---

The authors proceed to show where special cases of the InfoVAE objective return previously published models that attempt to balance the ELBO function for inference or information preference. It's easy to verify that $\alpha = 0$ and $\lambda = 1$ returns the original ELBO function. Choosing the simplex $\alpha + \lambda - 1 = 0$ (dropping out the mutual information term) gives the $\beta$-VAE family discussed in Higgins et al. (2017).

New to our class, but used as the primary counterexample in the paper, is the Maximum-mean discrepancy (MMD) divergence. The MMD is quite intuitive, given by

$$
    MMD[q(\cdot)\ ||\ p(\cdot)] = ||\mu_q - \mu_p||^2 = \mathbb{E}_{q(z),q(z')}[k(z,z')] - 2\mathbb{E}_{q(z), p(z')}[k(z, z')] + \mathbb{E}_{p(z), p(z')}[k(z,z')],
$$

where $k(z, z')$ is any positive-definite kernel between points $z$ and $z'$ sampled from the subscripted distributions in each expectation.

## 4.1. InfoVAE results

I show one comparison of samples generated from MNIST data using a standard VAE model compared with the proposed InfoVAE model. The images below are adapted from Figure 5 of Zhao et al. (2017).

<table>
    <tr>
        <td> <img src="elbo_samples_test.png" style="height:300px;"></td>
        <td> <img src="infovae_samples_test.png" style="height:300px"></td>
    </tr>
</table>

<center><strong>Figure 1</strong>. ELBO test (left) and MMD InfoVAE test samples (right). Notice that although the InfoVAE samples are dimmer they are also smoother.</center>

#  5. Conclusions


I wasn't able to get any code working with this model without using the `tensorflow` examples shared in the main article's repository, and didn't discover this until it was too late to discuss it. The fundamentally hard aspect of this for me was coding the reparameterization trick over two expectations in Equation $(5)$, which indicated a significant knowledge gap in what that notation actually means. Moreover, 

But my impression of the paper is that it is a meaningful extension for VAE models in general. My canonical examples for the project were going to be generating hierarchical regression models or trying to refit the kidney cancer data from Homework 3 using this model to see the results as a successful implementation of this model. For the failure, I would have liked to implement the same model in my Figure 1, but using a binary latent state with zeros and ones from the MNIST data. If this didn't work, my second choice would have been to use a binary latent variable state with a conditional sinusoidal/exponential regression problem. Since VAEs assume a continuous latent space, I suspect that we can use a highly discrete latent state to force model failure. Last, an alternative would be to use a low-dimensional sample space with a high-dimensional latent space.

# References

[1]. Zhao, S., Song, J. and Ermon, S., 2017. **Infovae: Information maximizing variational autoencoders**. arXiv preprint arXiv:1706.02262.

[2]. Kingma, D.P. and Welling, M., 2013. **Auto-encoding variational bayes//. arXiv preprint arXiv:1312.6114.

[3]. Doersch, C., 2016. **Tutorial on variational autoencoders**. arXiv preprint arXiv:1606.05908.

[4]. Goodfellow, I., Pouget-Abadie, J., Mirza, M., Xu, B., Warde-Farley, D., Ozair, S., Courville, A. and Bengio, Y., 2014. **Generative adversarial nets**. In Advances in neural information processing systems (pp. 2672-2680).

[5]. Makhzani, A., Shlens, J., Jaitly, N., Goodfellow, I. and Frey, B., 2015. **Adversarial autoencoders**. arXiv preprint arXiv:1511.05644.

[6]. Chen, X., Kingma, D.P., Salimans, T., Duan, Y., Dhariwal, P., Schulman, J., Sutskever, I. and Abbeel, P., 2016. **Variational lossy autoencoder**. arXiv preprint arXiv:1611.02731.

[7]. Higgins, I., Matthey, L., Pal, A., Burgess, C., Glorot, X., Botvinick, M., Mohamed, S. and Lerchner, A., 2017. **beta-VAE: Learning Basic Visual Concepts with a Constrained Variational Framework**. ICLR, 2(5), p.6.